In [1]:
import pandas as pd

In [2]:
# Define food and drinks sets
food = {'Add On', 'Bar Snack', 'Bites', 'Brunch', 'Burger', 'Chargrill', 'Chicken or Prawn Add ', 
        'Combo', 'Dessert', 'Doneness', 'Foods', 'GF Add Ons', 'GF Burger and Hotdog', 'Mains', 
        'Pasta', 'Pizza', 'Platter', 'Serving Choice', 'Sides', 'Smoothie', 'Soup', 'Starter', 'Hotdog'}

drinks = {'Bottled Beer', 'Bourbon', 'Choice of Margarita / ', 'Ciders', 'Cocktail', 'Coffee', 
          'Coffee Options', 'Cognac', 'Corkage', 'Draught Beers', 'Drinks', 'Gin', 'Juices', 
          'Liqueurs & Aperitifs', 'Live Craft Beer', 'Mineral Water', 'Mocktails', 'Red Wine', 
          'Rose Wine', 'Rum', 'Soft Drink', 'Sparkling & Champagne', 'Tea', 'Tequila', 'Vodka', 
          'Whisky', 'White Wine'}

# Date range boundaries
start_date = pd.Timestamp('2024-12-01')
end_date = pd.Timestamp('2025-03-31')

def clean_itemgroup_data(file_path):
    # Read Excel file
    df = pd.read_excel(file_path, engine='xlrd', skiprows=4)
    df = df[:-1]
    df = df.dropna(axis=1, how='all')
    df.columns = ['Date'] + list(df.columns[1:])
    
    # Drop unnecessary columns
    if 'Unnamed: 2' in df.columns:
        df = df.drop(columns=['Unnamed: 2'])
    
    df = df[df['Date'] != 'Total']
    df = df[df['Date'].notna()]
    
    # Drop columns with 'Unnamed' prefix
    unnamed_columns = [col for col in df.columns if col.startswith('Unnamed')]
    df = df.drop(columns=unnamed_columns)
    
    # Format the Date column
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
    
    # Filter by date range
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    # Convert the date back to the desired format
    df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
    
    # Convert all columns except 'Date' to float, handling non-numeric values
    for col in df.columns[1:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Calculate total_food and total_drinks for each row
    food_cols = list(food.intersection(df.columns))
    drink_cols = list(drinks.intersection(df.columns))

    df['total_food'] = df[food_cols].sum(axis=1, skipna=True)
    df['total_drinks'] = df[drink_cols].sum(axis=1, skipna=True)
    
    # Drop all columns except Date, total_food, total_drinks, Total
    columns_to_keep = ['Date', 'total_food', 'total_drinks', 'Total']
    df = df[[col for col in columns_to_keep if col in df.columns]]
    
    # Convert column names to lowercase
    df.columns = df.columns.str.lower()
    
    return df

def clean_daily_summary(file_path):
    # Read CSV file with appropriate skipping of rows and footers
    df = pd.read_csv(file_path, skiprows=4, skipfooter=7, engine='python')
    
    # Drop unnamed columns
    unnamed_columns = [col for col in df.columns if col.startswith('Unnamed:')]
    df.drop(columns=unnamed_columns, inplace=True)
    
    # Convert START column to date format
    df['START'] = pd.to_datetime(df['START'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
    
    # Filter by date range
    df = df[(df['START'] >= start_date) & (df['START'] <= end_date)]
    
    # Format the date to the desired format
    df['START'] = df['START'].dt.strftime('%d/%m/%Y')
    
    # Drop all other columns except START, ORDERS COUNT, AVG ORDER
    columns_to_keep = ['START', 'ORDERS COUNT', 'AVG ORDER']
    df = df[[col for col in columns_to_keep if col in df.columns]]
    
    # Rename START to date
    df.rename(columns={'START': 'date'}, inplace=True)
    
    # Convert all column names to lowercase
    df.columns = df.columns.str.lower()
    
    # Group by date and sum the values for duplicates
    df = df.groupby('date', as_index=False).sum()

    return df

def merge_outlet(itemgroup, daily_summary, outlet_name):
    # Merge itemgroup and daily summary using an outer join on date
    merged = pd.merge(itemgroup, daily_summary, on='date', how='outer', suffixes=('_itemgroup', '_daily'))
    # Sort by date
    merged = merged.sort_values(by='date').reset_index(drop=True)
    # Rename the merged DataFrame for each outlet
    merged.name = f"{outlet_name}_merged"
    return merged

In [3]:
df = pd.read_excel(r'daily_data\Picotin_overview.xlsx', engine='openpyxl')
df = df.drop(columns=['Timestamp'])
df = df.dropna(axis=1, how='all')  

ASQ_itemgroup = clean_itemgroup_data(r'daily_data\ASQ_itemgroup_daily.xls')
ASQ_daily_summary = clean_daily_summary(r'daily_data\ASQ_daily_summary.csv')

KATONG_itemgroup = clean_itemgroup_data(r'daily_data\KATONG_itemgroup_daily.xls')
KATONG_daily_summary = clean_daily_summary(r'daily_data\KATONG_daily_summary.csv')

RC_itemgroup = clean_itemgroup_data(r'daily_data\RC_itemgroup_daily.xls')
RC_daily_summary = clean_daily_summary(r'daily_data\RC_daily_summary.csv')

In [4]:
df

,Outlet,Date,Day,Breakfast Sales,Lunch Sales,Evening Sales,Dinner Lunch Sales,Night Sales,Total Sales,Number of Breakfast Floor Staff,...,Number of Lunch Kitchen Staff,Number of Evening Kitchen Staff,Number of Dinner Kitchen Staff,Number of Night Kitchen Staff,Total Taxi Claims,Last Bill Closed,Closing Time,Closing Manager Name,Total Floor Staff Hours,Total Kitchen Staff Hours
0,Asia Square,2024-11-01 00:00:00,Fri,0.0,693.03,373.55,953.09,0.00,2019.67,1.0,...,3.0,3.0,3.0,0.0,0.0,22:01:00,22:30:00,RACQUEL,26.5,36.0
1,Asia Square,2024-11-04 00:00:00,Mon,0.0,898.89,199.03,922.01,0.00,2019.93,1.0,...,4.0,4.0,4.0,0.0,0.0,22:00:00,22:30:00,Racquel,31.5,52.5
2,Asia Square,2024-11-05 00:00:00,Tue,0.0,1811.05,545.55,1404.04,136.68,3897.32,1.0,...,4.0,4.0,4.0,0.0,0.0,22:45:00,23:00:00,Racquel,34.0,53.0
3,Asia Square,2024-11-06 00:00:00,Wed,0.0,1263.75,722.99,1447.63,157.07,3591.44,1.0,...,4.0,4.0,4.0,0.0,0.0,22:45:00,23:00:00,RACQUEL,33.0,53.0
4,Asia Square,2024-11-07 00:00:00,Thu,0.0,678.65,490.39,2553.87,33.57,3756.48,1.0,...,4.0,4.0,4.0,0.0,0.0,22:45:00,23:00:00,RACQUEL,34.0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,Asia Square,2025-04-07 00:00:00,Mon,0.0,1182.93,398.07,1129.47,0.00,2710.47,1.0,...,4.0,4.0,4.0,0.0,0.0,22:26:00,23:00:00,Mathew,41.5,46.0
473,Katong,2025-04-07 00:00:00,Mon,0.0,0.00,160.23,800.95,522.77,1483.95,0.0,...,1.0,3.0,3.0,2.0,0.0,10:30:00,11:30:00,SunShine :)),19.0,18.5
474,Rochester,2025-04-08 00:00:00,Tue,0.0,0.00,356.40,2541.00,0.00,2897.40,0.0,...,2.0,2.0,2.0,0.0,0.0,10:15:00,10:30:00,Aswathy,27.0,21.0
475,Asia Square,2025-04-08 00:00:00,Tue,0.0,1830.91,408.87,2049.08,0.00,4288.86,1.0,...,4.0,4.0,4.0,0.0,0.0,22:28:00,23:00:00,Mathew,38.0,47.0


In [5]:
RC_itemgroup

,date,total_food,total_drinks,total
12,01/12/2024,494.8,602.0,1096.8
13,02/12/2024,1810.0,1288.0,3104.0
14,03/12/2024,2425.4,1656.0,4081.4
16,04/12/2024,1561.0,1210.0,2771.0
17,05/12/2024,1193.8,1253.0,2732.8
...,...,...,...,...
147,27/03/2025,948.0,1091.0,2039.0
149,28/03/2025,1180.0,999.0,3131.0
150,29/03/2025,506.5,737.0,1243.5
151,30/03/2025,840.0,600.0,1440.0


In [6]:
RC_daily_summary

,date,orders count,avg order
0,01/01/2025,15,73.44
1,01/02/2025,22,36.79
2,01/03/2025,43,63.61
3,02/01/2025,23,64.66
4,02/02/2025,26,47.48
...,...,...,...
111,30/01/2025,27,102.85
112,30/03/2025,60,63.41
113,30/12/2024,19,63.91
114,31/01/2025,57,56.39


In [7]:
outlets = ['ASQ', 'KATONG', 'RC']

for outlet in outlets:
    # Access the respective DataFrames dynamically
    daily_summary = globals().get(f'{outlet}_daily_summary')
    itemgroup = globals().get(f'{outlet}_itemgroup')
    
    if daily_summary is None or itemgroup is None:
        print(f"\nWarning: DataFrames for {outlet} are not loaded.")
        continue

    # Find unique dates from both DataFrames
    summary_dates = set(daily_summary['date'].dropna().unique())
    itemgroup_dates = set(itemgroup['date'].dropna().unique())

    # Find missing dates
    missing_in_itemgroup = summary_dates - itemgroup_dates
    missing_in_summary = itemgroup_dates - summary_dates

    # Find duplicate dates in each DataFrame
    duplicates_in_summary = daily_summary['date'][daily_summary['date'].duplicated()]
    duplicates_in_itemgroup = itemgroup['date'][itemgroup['date'].duplicated()]

    # Display the results for the current outlet
    print(f"\nOutlet: {outlet}")
    print("Dates missing in itemgroup:", sorted(missing_in_itemgroup))
    print("Dates missing in daily summary:", sorted(missing_in_summary))
    print("Duplicate dates in daily summary:", duplicates_in_summary.unique())
    print("Duplicate dates in itemgroup:", duplicates_in_itemgroup.unique())



Outlet: ASQ
Dates missing in itemgroup: []
Dates missing in daily summary: []
Duplicate dates in daily summary: []
Duplicate dates in itemgroup: []

Outlet: KATONG
Dates missing in itemgroup: []
Dates missing in daily summary: ['31/03/2025']
Duplicate dates in daily summary: []
Duplicate dates in itemgroup: []

Outlet: RC
Dates missing in itemgroup: []
Dates missing in daily summary: ['01/12/2024', '05/12/2024', '18/01/2025', '24/12/2024', '31/03/2025']
Duplicate dates in daily summary: []
Duplicate dates in itemgroup: []


In [8]:
# Merge each outlet
ASQ_merged = merge_outlet(ASQ_itemgroup, ASQ_daily_summary, 'ASQ')
KATONG_merged = merge_outlet(KATONG_itemgroup, KATONG_daily_summary, 'KATONG')
RC_merged = merge_outlet(RC_itemgroup, RC_daily_summary, 'RC')

In [9]:
# Add the outlet name as a column to each merged DataFrame
ASQ_merged['outlet'] = 'Asian Square'
KATONG_merged['outlet'] = 'Katong'
RC_merged['outlet'] = 'Rochester'

# Concatenate the three merged DataFrames while preserving all data (outer join)
combined_merged = pd.concat([ASQ_merged, KATONG_merged, RC_merged], axis=0, ignore_index=True)

# Sort by outlet and date for better organization
combined_merged = combined_merged.sort_values(by=['outlet', 'date']).reset_index(drop=True)

In [10]:
combined_merged.head()

,date,total_food,total_drinks,total,orders count,avg order,outlet
0,02/01/2025,386.0,276.0,662.0,36.0,22.05,Asian Square
1,02/12/2024,1246.0,957.0,2210.0,60.0,44.16,Asian Square
2,03/01/2025,1177.0,1307.0,2484.0,49.0,48.45,Asian Square
3,03/02/2025,979.0,725.0,1738.0,59.0,35.31,Asian Square
4,03/03/2025,1367.8,763.0,2139.8,64.0,40.05,Asian Square


In [ ]:
# Mapping of outlet codes to full names
outlet_mapping = {
    'ASQ': 'Asia Square',
    'KATONG': 'Katong',
    'RC': 'Rochester'
}

# Add Outlet column to each merged DataFrame
ASQ_merged['outlet'] = outlet_mapping['ASQ']
KATONG_merged['outlet'] = outlet_mapping['KATONG']
RC_merged['outlet'] = outlet_mapping['RC']

# Fix the date format in each merged DataFrame
ASQ_merged['date'] = pd.to_datetime(ASQ_merged['date'], format='%m/%d/%Y', errors='coerce').dt.strftime('%Y-%m-%d')
RC_merged['date'] = pd.to_datetime(RC_merged['date'], format='%m/%d/%Y', errors='coerce').dt.strftime('%Y-%m-%d')
KATONG_merged['date'] = pd.to_datetime(KATONG_merged['date'], format='%m/%d/%Y', errors='coerce').dt.strftime('%Y-%m-%d')

# Fix the date format in the main DataFrame
df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Combine all merged DataFrames into one
merged_outlets = pd.concat([ASQ_merged, KATONG_merged, RC_merged], ignore_index=True)

# Standardize Outlet names
merged_outlets['outlet'] = merged_outlets['outlet'].str.strip().str.title()
df['Outlet'] = df['Outlet'].str.strip().str.title()

# Merge the combined merged_outlets DataFrame with the main df on Outlet and Date
final_merged = pd.merge(df, merged_outlets, left_on=['Outlet', 'Date'], right_on=['outlet', 'date'], how='outer')

# Drop redundant 'outlet' and 'date' columns from the merged result
final_merged = final_merged.drop(columns=['outlet', 'date'])

# Sort the final result by Outlet and Date
final_merged = final_merged.sort_values(by=['Outlet', 'Date']).reset_index(drop=True)

final

,Outlet,Date,Day,Breakfast Sales,Lunch Sales,Evening Sales,Dinner Lunch Sales,Night Sales,Total Sales,Number of Breakfast Floor Staff,...,Last Bill Closed,Closing Time,Closing Manager Name,Total Floor Staff Hours,Total Kitchen Staff Hours,total_food,total_drinks,total,orders count,avg order
0,Asia Square,2024-10-01,Tue,0.0,1498.79,581.53,1249.37,570.72,3900.41,1.0,...,22:33:00,23:00:00,RACQUEL,34.5,36.0,NaN,NaN,NaN,NaN,NaN
1,Asia Square,2024-10-02,Wed,0.0,1505.34,585.12,3526.75,47.72,5664.93,1.0,...,22:42:00,23:00:00,RACQUEL,33.5,36.0,NaN,NaN,NaN,NaN,NaN
2,Asia Square,2024-10-03,Thu,0.0,1178.61,778.17,1745.75,0.00,3702.53,1.0,...,22:15:00,22:30:00,RACQUEL,41.0,36.0,NaN,NaN,NaN,NaN,NaN
3,Asia Square,2024-10-04,Fri,0.0,978.49,518.01,2190.57,369.29,4056.36,1.0,...,22:38:00,23:00:00,RACQUEL,33.5,36.0,NaN,NaN,NaN,NaN,NaN
4,Asia Square,2024-10-07,Mon,0.0,701.97,287.60,1072.74,134.29,2196.60,1.0,...,22:30:00,23:00:00,RACQUEL,24.5,41.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,840.0,600.0,1440.0,60.0,63.41
906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,484.0,625.0,1109.0,19.0,63.91
907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1977.0,2404.0,4381.0,57.0,56.39
908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1095.0,940.0,2035.0,NaN,NaN


In [12]:
print("ASQ_merged dates:", ASQ_merged['date'].head())
print("RC_merged dates:", RC_merged['date'].head())
print("KATONG_merged dates:", KATONG_merged['date'].head())
print("Main df dates:", df['Date'].head())
print("ASQ_merged outlet:", ASQ_merged['outlet'].unique())
print("RC_merged outlet:", RC_merged['outlet'].unique())
print("KATONG_merged outlet:", KATONG_merged['outlet'].unique())
print("Main df outlet:", df['Outlet'].unique())


ASQ_merged dates: 0    2025-02-01
1    2024-02-12
2    2025-03-01
3    2025-03-02
4    2025-03-03
Name: date, dtype: object
RC_merged dates: 0    2025-01-01
1    2025-01-02
2    2025-01-03
3    2024-01-12
4    2025-02-01
Name: date, dtype: object
KATONG_merged dates: 0    2025-01-01
1    2025-01-02
2    2025-01-03
3    2024-01-12
4    2025-02-01
Name: date, dtype: object
Main df dates: 0    2024-11-01
1    2024-11-04
2    2024-11-05
3    2024-11-06
4    2024-11-07
Name: Date, dtype: object
ASQ_merged outlet: ['Asia Square']
RC_merged outlet: ['Rochester']
KATONG_merged outlet: ['Katong']
Main df outlet: ['Asia Square' 'Rochester' 'Katong']
